In [ ]:
import pandas as pd
import os, time
import numpy as np
from tensorflow.keras.applications import ResNet50
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_model_optimization as tfmot
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
#ResNet50
data_dir = "data/north_american_predators/original_downsampled_split/train"
dataset_train = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)

data_dir = "data/north_american_predators/original_downsampled_split/val"
dataset_val = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)

data_dir = "data/north_american_predators/original_downsampled_split/test"
dataset_test = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)
print(dataset_train.class_names)

Found 13823 files belonging to 8 classes.
Found 3454 files belonging to 8 classes.
Found 4317 files belonging to 8 classes.
['black_bear', 'brown_bear', 'cougar', 'coyote', 'human', 'non_human', 'polar_bear', 'wolf']


In [ ]:
def evaluate_model_tflite(interpreter, dataset):
    #List of dictionaries
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    correct = 0
    total = 0

    for images, labels in dataset:
        input_type = input_details[0]['dtype']
        # Convert the batch to the correct type
        images = images.numpy().astype(input_type)
        labels = labels.numpy()
        
        for i in range(images.shape[0]):
            test_image = np.expand_dims(images[i], axis=0)
            #From Tensorflow API Docs
            interpreter.set_tensor(input_details[0]['index'], test_image)
            interpreter.invoke()
            output = interpreter.get_tensor(output_details[0]['index'])
            #Gets the most prominent class
            pred = np.argmax(output)
            if pred == labels[i]:
                correct += 1
            total += 1

    accuracy = correct / total
    return accuracy

In [4]:
base_model = tf.keras.applications.ResNet50(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
outputs = tf.keras.layers.Dense(8, activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

In [5]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
early_stop = EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    patience=5,
    restore_best_weights=True,
    mode='max'
)

start_time = time.time()
model.fit(dataset_train, epochs=100, validation_data=dataset_val, callbacks=[early_stop])
end_time = time.time()
training_duration = end_time - start_time
print(f"Training time: {training_duration:.2f} seconds")

Epoch 1/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 186s 853ms/step - loss: 0.2113 - sparse_categorical_accuracy: 0.9348 - val_loss: 0.4134 - val_sparse_categorical_accuracy: 0.8590
Epoch 2/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 179s 831ms/step - loss: 0.0566 - sparse_categorical_accuracy: 0.9833 - val_loss: 0.3700 - val_sparse_categorical_accuracy: 0.8825
Epoch 3/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 176s 815ms/step - loss: 0.0400 - sparse_categorical_accuracy: 0.9871 - val_loss: 0.3932 - val_sparse_categorical_accuracy: 0.8836
Epoch 4/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 177s 818ms/step - loss: 0.0288 - sparse_categorical_accuracy: 0.9917 - val_loss: 0.3528 - val_sparse_categorical_accuracy: 0.8958
Epoch 5/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 175s 811ms/step - loss: 0.0236 - sparse_categorical_accuracy: 0.9933 - val_loss: 0.3713 - val_sparse_categorical_accuracy: 0.8937
Epoch 6/100
216/216 ━━━━━━━━━━━━━━━━━━━━ 175s 812ms/step - loss: 0.0190 - sparse_categorical_accuracy: 0.9948 - val_loss: 0.3441 - val_sparse_cat

In [6]:
start_time = time.time()
test_loss, test_acc = model.evaluate(dataset_test, verbose=0)
end_time = time.time()
training_duration = end_time - start_time
print(f"Evaluation time: {training_duration:.2f} seconds")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc*100:.2f}%")

Evaluation time: 44.20 seconds
Test Loss: 0.2565
Test Accuracy: 92.24%


In [7]:
model.save('model_ResNet50_original.keras')
print("Baseline model in Mb:", os.path.getsize('model_ResNet50_original.keras') / float(2**20))

Baseline model in Mb: 90.78868198394775


In [8]:
loaded_model = tf.keras.models.load_model("model_ResNet50_original.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open('model_ResNet50_quantized_original.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmpz9sonyml\assets


INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmpz9sonyml\assets


Saved artifact at 'C:\Users\kingk\AppData\Local\Temp\tmpz9sonyml'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  2833292675600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2833292675024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2833292674832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2833292676944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2833292676752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2833292676176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2833292676560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2833292673296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2833292675216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2833292678096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  283329

In [9]:
loaded_model = tf.keras.models.load_model("model_ResNet50_original.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()
with open('model_ResNet50_original.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmp8dyi2qo3\assets


INFO:tensorflow:Assets written to: C:\Users\kingk\AppData\Local\Temp\tmp8dyi2qo3\assets


Saved artifact at 'C:\Users\kingk\AppData\Local\Temp\tmp8dyi2qo3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  2832988156752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2835600081360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2835600081168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2835872906512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2835600080976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2835600082704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2835872906896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2835872906128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2835872906704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2835872904976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  283587

In [ ]:
#ResNet50 trained on original images
interpreter = tf.lite.Interpreter(model_path="model_ResNet50_original.tflite")
interpreter.allocate_tensors()
acc = evaluate_model_tflite(interpreter, dataset_test)        
print("TFLite accuracy:", acc)
print("TFlite model in Mb:", os.path.getsize('model_ResNet50_original.tflite') / float(2**20))

c:\Graduate_School\Fall_2025\ECE528\.venv\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


TFLite accuracy: 0.9223998146861246
TFlite model in Mb: 89.66934967041016


In [ ]:
#ResNet50 quantized trained on original images
interpreter = tf.lite.Interpreter(model_path="model_ResNet50_quantized_original.tflite")
interpreter.allocate_tensors()
acc = evaluate_model_tflite(interpreter, dataset_test)
print("TFLite accuracy:", acc)
print("TFlite model in Mb:", os.path.getsize('model_ResNet50_quantized_original.tflite') / float(2**20))

TFLite accuracy: 0.9228630993745657
TFlite model in Mb: 22.82269287109375
